In [1]:
import iris
import xarray as xr
import numpy as np
from iris.coords import DimCoord
from iris.coord_systems import TransverseMercator,GeogCS
from iris.cube import Cube
from cf_units import Unit
import cf_units
import os
import glob
from pyproj import Proj, transform
import sys

############################################
# Define variables and set up environment
#############################################
root_fp = "/nfs/a319/gy17m2a/PhD"
os.chdir(root_fp)


In [2]:
year = 2013

In [3]:
# Load UKCP18 model data to use in regriddding
file_model_2_2km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_19911201-19911230.nc'
cube_model_2_2km =iris.load_cube(file_model_2_2km)

# Load UKCP18 12km model data to use in regriddding
file_model_12km ='/nfs/a319/gy17m2a/PhD/datadir/UKCP18/12km/01/pr_rcp85_land-rcm_uk_12km_01_day_19801201-19901130.nc'
cube_model_12km=iris.load_cube(file_model_12km)

### Make necessary folders to store regridded data

In [4]:
folder_regrid_2_2km_nn

NameError: name 'folder_regrid_2_2km_nn' is not defined

In [5]:
time_resolution  = '30mins'
for year in range(2013,2020):
    print(year)
    folder_regrid_2_2km_lin = (f"datadir/NIMROD/{time_resolution}/OriginalFormat_1km/{year}").replace('OriginalFormat_1km', 'NIMROD_regridded_2.2km/LinearRegridding')
    folder_regrid_2_2km_nn = (f"datadir/NIMROD/{time_resolution}/OriginalFormat_1km/{year}").replace('OriginalFormat_1km', 'NIMROD_regridded_2.2km/NearestNeighbour')

    folder_regrid_12km_lin = (f"datadir/NIMROD/{time_resolution}/OriginalFormat_1km/{year}").replace('OriginalFormat_1km', 'NIMROD_regridded_12km/LinearRegridding')
    folder_regrid_12km_nn = (f"datadir/NIMROD/{time_resolution}/OriginalFormat_1km/{year}").replace('OriginalFormat_1km', 'NIMROD_regridded_12km/NearestNeighbour')
    print("Making folders")
    
    for folder_fp in [folder_regrid_2_2km_lin, folder_regrid_2_2km_nn, folder_regrid_12km_lin, folder_regrid_12km_nn]:

        if not os.path.isdir(folder_fp):
            os.makedirs(folder_fp)

2013
Making folders
2014
Making folders
2015
Making folders
2016
Making folders
2017
Making folders
2018
Making folders
2019
Making folders


### For each file in the NimRod directory:
#### Reformat and then regrid into same format as the UKCP18 model cube

In [8]:
cube =iris.load_cube("datadir/20140601_nimrod_ng_radar_rainrate_composite_1km_merged.nc")
cube[0]

OSError: One or more of the files specified did not exist:
    * "/nfs/a319/gy17m2a/PhD/datadir/20140601_nimrod_ng_radar_rainrate_composite_1km_merged.nc" didn't match any files

In [7]:
cube =iris.load_cube(filename)
cube

Rain Rate Composite (mm/hr),time,projection_y_coordinate,projection_x_coordinate
Shape,283,72,82
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
forecast_reference_time,x,-,-
Scalar coordinates,,,forecast_period 0 second
Attributes,,,Conventions CF-1.7field_code 213institution Met Officenimrod_version 2probability_period_of_event 0source 1km Single Site Radarstitle Unknown


In [5]:
for filename in glob.glob(f"datadir/NimRod/OriginalFormat/{year}/*"):
    # Load data
    cube =iris.load_cube(filename)
    
    for regridding_resolution in ['2.2km', '12km']:
        # Specify model cube to use in regridding
        if regridding_resolution == '2.2km':
            cube_model = cube_model_2_2km
        elif regridding_resolution == '12km':
            cube_model = cube_model_12km

        #### Regrid observaitons onto model grid
        # Lienar interpolation
        reg_cube_lin =cube.regrid(cube_model,iris.analysis.Linear())      
        # Nearest neighbour
        reg_cube_nn =cube.regrid(cube_model,iris.analysis.Nearest())    
        
        # Filename to save regridded cube to - 2.2km -- linear/nearest neighbour
        filename_regrid_lin = filename.replace(f"OriginalFormat/{year}/", f"NIMROD_regridded_{regridding_resolution}/LinearRegridding/{year}/rg_")
        filename_regrid_nn = filename.replace(f"OriginalFormat/{year}/", f"NIMROD_regridded_{regridding_resolution}/NearestNeighbour/{year}/rg_")

        # Save regridded
        iris.save(reg_cube_lin, filename_regrid_lin)
        iris.save(reg_cube_nn, filename_regrid_nn)
